# Librerias

In [1]:
from sklearn.cluster import KMeans as km
from sklearn.metrics import mean_squared_error as mse
from sklearn.datasets import make_blobs
import random
from sklearn.metrics import adjusted_rand_score as ari
from sklearn.metrics import completeness_score as completitud
from sklearn.metrics import fowlkes_mallows_score as FM
from sklearn.metrics.pairwise import euclidean_distances
import copy
from sklearn.metrics import davies_bouldin_score as DB
from sklearn.cluster import AgglomerativeClustering
import numpy as np

# Código PSOkmeans

In [ ]:
class particula:
  def __init__(self, d : int, x_min : float, x_max : float,
               v_min : float, v_max: float, k, centros):
    self.posicion = centros
    self.velocidad = np.random.uniform(v_min,v_max,size = (k,d))
    self.aptitud = None
    self.mejorPos = copy.deepcopy(self.posicion)
    self.apt_best = None
    self.nGrupos = k
    self.dim = d
    self.labels_ = None

In [ ]:
def aptitud(particula, datos):
  '''Asigna el valor de la aptitud a una particula en
  base a su particion y centroides '''
  #calculamos la aptitud
  separacion = 0
  for i,centro in enumerate(particula.posicion):
    #obtenemos los elementos del grupo i
    elementos = datos[np.where(particula.labels_ == i)]
    separacion+=sum(
        [np.linalg.norm(dato-centro)**2 for dato in elementos])
  return(separacion/len(datos))

In [ ]:
def aptitud2(particula,datos):
  return(DB(datos,particula.labels_))

In [ ]:
def iniciar(x_min,x_max,v_min,v_max,k,n_part,datos,f_apt):
  '''d es la dimension de los datos
  x_max y x_min delimitan el intervalo de los valores de los datos'''
  dim = len(datos[0])
  particulas= []
  for i in range(n_part):
    centros = np.array(random.choices(datos,k=k))
    part = particula(dim,x_min,x_max,v_max,v_min,k,centros)
    verificar_posicion(part,x_min,x_max,datos)
    part.aptitud = f_apt(part,datos)
    part.apt_best = copy.deepcopy(part.aptitud)
    particulas.append(part)
  mejor = copy.deepcopy(min(particulas, key=lambda x: x.aptitud))
  return(particulas,mejor)

In [ ]:
def mover_part(part,inercia,mejor_pos,c1,c2,x_min,x_max,v_min,v_max):
  r1,r2 = np.random.uniform(0,1,2)
  velocidad = inercia*part.velocidad+(c1*r1*(part.mejorPos-part.posicion))+(c2*r2*(mejor_pos-part.posicion))
  posicion = part.posicion + velocidad
  #revisar velocidad este dentro del rango
  velocidad[np.where(velocidad<v_min)] = v_min
  velocidad[np.where(velocidad>v_max)] = v_max
  return(velocidad,posicion)

In [ ]:
def verificar_posicion(particula,x_min,x_max, datos):
  ''' esta funcion verifica que los centroides no se salgan del rango del
  problema en cuestion, se considera que un centroide esta fuera de rango
  cuando (1): sus componentes salen del minimo o maximo valor permitido
  o (2): cuando el centroide no tienen ningun dato asignado'''
  for i,pos in enumerate(particula.posicion):
    #comprobamos si algun valor se sale de su x_min o de su x_max
    if np.where(pos<x_min)[0].size >0 or np.where(pos>x_max)[0].size >0:
      particula.posicion[i] = random.choice(datos)
  cambios = True
  while cambios:
    distancias  = euclidean_distances(datos,particula.posicion)
    etiquetas = np.argmin(distancias, axis = 1).tolist()
    cambios = False
    for i in range(particula.nGrupos):
      if etiquetas.count(i) == 0:
        particula.posicion[i] = random.choice(datos)
        cambios = True
  particula.labels_ = np.array(etiquetas)

In [ ]:
def pso(x_min,x_max,v_min,v_max,k,n_part,iter_pso,iter_kmean,c1,c2,datos,f_apt):
  '''n_part es el numero de particulas
  k es el numero de grupos a hacer
  x_min y x_max son vectores con los minimos y maximos valores
  de cada componente en los datos'''
  enjambre,mejor_inicial = iniciar(x_min=x_min,x_max=x_max,
                                   v_min=v_min,v_max=v_max,k=k,
                                   n_part=n_part, datos=datos,f_apt=f_apt)
  cont = 0; sin_cambios = 0; inercia = 0.6; dism = 0.5/(iter_pso-1)
  while True:
    if cont == iter_pso:
      break
    for part in enjambre: #actualizar posicion y velocidad
      part.velocidad,part.posicion = mover_part(part=part,inercia=inercia,
                                 mejor_pos = mejor_inicial.posicion,
                                 c1=c1, c2=c2, x_min=x_min, x_max=x_max,
                                 v_min=v_min, v_max=v_max)
      verificar_posicion(part,x_min,x_max,datos)
      part.aptitud = f_apt(part,datos)
      #verificamos la mejor posicion visitada de la particula
      if part.aptitud < part.apt_best:
        part.apt_best = copy.copy(part.aptitud)
        part.mejorPos = copy.copy(part.posicion)
    #obtener mejor posicion de la iteracion
    mejor_final = copy.deepcopy(min(enjambre, key= lambda x: x.aptitud))
    #inercia = inercia - dism
    cont += 1
    if mejor_final.aptitud < mejor_inicial.aptitud:
      mejor_inicial = copy.deepcopy(mejor_final)
      sin_cambios = 0
    else:
      sin_cambios += 1
    if sin_cambios == 10:
      break
  #ejecutamos kmeans
  reskmeans = km(n_clusters=k,n_init=1,
              max_iter=iter_kmean,init=mejor_inicial.posicion).fit(datos)
  reskmeans.posicion = reskmeans.cluster_centers_
  reskmeans.aptitud = f_apt(reskmeans,datos)
  return(reskmeans)